<a href="https://colab.research.google.com/github/dladutjs66/web-platform-tests/blob/master/var_crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## 필요 라이브러리 설치
!pip install currency-symbols
!pip install PublicDataReader
!pip install finance-datareader
!pip install pandas-datareader

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.3/89.3 kB 1.9 MB/s eta 0:00:00


In [2]:
### 라이브러리 import"""
import pandas as pd
from tabulate import tabulate
from datetime import datetime
from PublicDataReader import Ecos
import pandas_datareader as pdr
from datetime import datetime, timedelta
from FinanceDataReader.investing.data import InvestingDailyReader
from tqdm import tqdm
import time
from currency_symbols import CurrencySymbols

In [3]:
### 날짜 지정
start_date = '2015-01-01'
end_date = '2024-11-26'

In [4]:
### BOK 변수 크롤링
### bok_list에 bok 변수 명으로 이루어진 list 저장
#BOK
m1_bok_list = ["BOK$521Y001$A001$D" , "BOK$817Y002$010503000$D" , "BOK$731Y001$0000036$D" , "BOK$731Y001$0000035$D" , "BOK$731Y001$0000021$D" , "BOK$731Y002$0000039$D"]
m4_bok_list = ["BOK$731Y001$0000016$D","BOK$817Y002$010102000$D",'BOK$731Y002$0000039$D','BOK$731Y002$0000047$D']

bok_list = m4_bok_list.copy()

#3이 1530 종가 , 2가 시가, 5가 고가, 4가 저가
bok_code_list = [i.split('$')[1] for i in bok_list]
bok_code1_list = [i.split('$')[2] for i in bok_list]
bok_freq_list = [i.split('$')[3] for i in bok_list]

service_key = 'SD6GBGAU9S8F64JB2RQB'
api = Ecos(service_key)
dfs=[]
for i in zip(bok_code_list,bok_code1_list,bok_freq_list):
    code = i[0]
    code1 = i[1]
    freq = i[2]
    df = api.get_statistic_search(통계표코드=code,통계항목코드1=code1 ,주기=freq, 검색시작일자="20150101", 검색종료일자=end_date.replace('-',''))
    df = df[['시점','값']]
    df['시점'] = pd.to_datetime(df['시점'], format='%Y%m%d')
    df = df.set_index('시점')
    df.columns = ['BOK$' + '$'.join([code,code1,freq])]
    dfs.append(df)
bok_dataset = pd.concat(dfs,axis=1)

bok_dataset.to_csv('bok_m4.csv',encoding='cp949')

In [5]:
### FRED 변수 크롤링
### fred_list에 fred 변수명으로 이루어진 list 저장

fred_list = ["FRED$RIFSPPNA2P2D15NB_VALUE" , "FRED$T5YIE_VALUE"]

dfs=[]
for i in fred_list:
    i_repl = i.replace('FRED$','')
    i_repl = i_repl.replace('_VALUE','')
    df = pdr.get_data_fred(i_repl,start=datetime(2015,1,1),end=pd.to_datetime(end_date).date())
    df.columns=[i]
    dfs.append(df)

fred_df = pd.concat(dfs,axis=1)

fred_df.to_csv('fred_m4.csv',encoding='cp949')

In [8]:
### INV 변수 크롤링
### inv_ls에 inv 변수명으로 이루어진 list 저장
### 변수명 전처리 과정으로, 필요한 전처리 규칙이 있으면 j=j.replace()에 삽입

inv_ls_m4 = ["INV$300618_CLOSE","INV$EURMXN_CLOSE","INV$CLC3_VOLUME","INV$PL1MTRR_CLOSE","INV$600415_CLOSE","INV$BR1YTXX_CLOSE","INV$SMC3_CLOSE","INV$TFMBMC1_CLOSE","INV$HOC1_VOLUME","INV$TFMBMC1_VOLUME","INV$LHOC3_VOLUME","INV$HOC2_VOLUME"]
inv_ls=inv_ls_m4.copy()
new_inv_ls=[]
col_change_dict = {}
for j in inv_ls:
    org_j = j
    j = j.replace('LBC','LXRC')
    j = j.replace('LBC','LXRC')
    j = j.replace('NYF','NYFV4')
    j = j.replace('C1','c1')
    j = j.replace('C2','c2')
    j = j.replace('C3','c3')
    j = j.replace('YTRR','YT=RR')
    j = j.replace('MTXX','MT=XX')
    j = j.replace('MTRR','MT=RR')
    j = j.replace('MTX','MT=X')
    j = j.replace('YTXX','YT=XX')
    j = j.replace('XAGTDSGEX','XAGTD=SGEX')
    j_first_three = j.split('_')[0].split('$')[1][:3]
    j_last_three = j.split('_')[0].split('$')[1][-3:]
    if len(j.split('_')[0].split('$')) == 3:
      add_str = j.split('_')[0].split('$')[-1]
    else:
      add_str = 'None'
    if_currency_first = CurrencySymbols.get_symbol(j_first_three)
    if_currency_last = CurrencySymbols.get_symbol(j_last_three)
    if ((type(if_currency_first) == str) & (type(if_currency_last) == str)) | (j_first_three=='WTI'):
      if add_str != 'None':
          j = 'INV$'+j_first_three + '/' + j_last_three+' '+add_str+'_'+j.split('_')[-1]
      else:
          j = 'INV$'+j_first_three + '/' + j_last_three+'_'+j.split('_')[-1]
    else:
        pass
    col_change_dict[j] = org_j
    new_inv_ls.append(j)

In [9]:
### INV 변수를 다운받는 과정

end = datetime.now().strftime("%Y-%m-%d")
start='2015-01-01'
dfs=[]
for i in tqdm(new_inv_ls):
    inv_repl  = i.replace('INV$','')
    wanted_col = inv_repl.split('_')[1]
    search_col = inv_repl.split('_')[0]
    reader = InvestingDailyReader(symbol=search_col, start=start, end=end_date)
    try:
        data = reader.read()
        data.columns = [i.upper() for i in data.columns]
        data = data[[wanted_col]]
        data.columns=[i]
        dfs.append(data)
        time.sleep(0.1)
    except:
        continue

inv_df = pd.concat(dfs,axis=1)
#inv_df.columns = [i.replace('/','').replace('=','').upper() for i in inv_df.columns]
inv_df.columns = [col_change_dict[i] for i in inv_df.columns]

inv_df.to_csv('inv_m4.csv',encoding='cp949')

100%|██████████| 12/12 [01:54<00:00,  9.51s/it]
